# Get transients given a skymap

- Calculate skymap volume. Calculate the fraction of total surface area covered by a given voxel (skymap volume pixel), and multiply that fraction by the volume of a sphere with radius 1000 Mpc

- Use rates from literature to draw a sample of transients given the skymap volume


to do - 

add : poisson sampled distribution

work in redshift not dl ?

use galactic latitude (cvs)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from io import BytesIO
import xmltodict
import requests
import os

from astropy.table import Table
from astropy.table import QTable
from astropy.io import fits
import astropy_healpix as ah
import astropy.units as u
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess.cosmology import dVC_dVL_for_DL
from astropy.cosmology import Planck15 as cosmo, z_at_value

from ligo.gracedb.rest import GraceDb
g = GraceDb()

In [23]:
# load skymaps from local directory

def load_skymap_fits(filename, directory):
    filepath = os.path.join(directory, filename)
    with fits.open(filepath) as hdul:
        table = Table.read(hdul)
        return table
    
# load simulated skymaps    
directory = 'skymaps/nsbh_allsky/'
skymap_sim = [load_skymap_fits(file, directory) for file in os.listdir(directory) if file.endswith('.fits')]
names_sim = [os.path.splitext(file)[0] for file in os.listdir(directory) if file.endswith('.fits')]

# load gracedb skymaps
directory = 'skymaps/gracedb/'
skymap_gracedb = [load_skymap_fits(file, directory) for file in os.listdir(directory) if file.endswith('.fits')]
names_gracedb = [os.path.splitext(file)[0] for file in os.listdir(directory) if file.endswith('.fits')]

In [20]:
# adapted from https://github.com/lpsinger/ligo.skymap/blob/a8da314dcb078f7866f4178cbd523b9844cceae0/ligo/skymap/postprocess/crossmatch.py

def get_skymap_voxels(skymap, name, contour=0.9, cosmology=True):
    """
    Determine the volume per pixel in the 90% region of a skymap
    This volume is associated with a sky position and can be used with positionally nonuniform rates
    """
    # get specified countour of skymap, drop other pixels
    skymap.sort('PROBDENSITY', reverse = True)
    level, ipix = ah.uniq_to_level_ipix(skymap['UNIQ']) # ipix = sky location
    nside = ah.level_to_nside(level) # nside = multi-order pixel resolution
    pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level)) # pixel area in steradians
    prob = pixel_area * skymap['PROBDENSITY']
    cumprob = np.cumsum(prob)
    i = cumprob.searchsorted(contour)
    dA_sr = pixel_area[:i]

    # Calculate volume of each voxel, defined as the region within the
    # HEALPix pixel and contained within the two centric spherical shells
    # with radii (r - d_r / 2) and (r + d_r / 2).
    # dA_deg = dA_sr.to_value(u.deg**2) #areas in deg**2 per pixel for only pixels in countour
    # r = 500 #mpc
    # d_r = 1000 #mpc
    # dV = (np.square(r) + np.square(d_r) / 12) * d_r * dA_deg.reshape(-1, 1) #naive euclidean volume from LIGO code

    # replacing above with my own Voxel - multiply volume of sphere radius 1000Mpc by fraction of pixel steradians / 4pi steradians in a sphere
    r = 1000 #Mpc
    dV = 4/3 * np.pi * r**3 * dA_sr.reshape(-1, 1)/(4 * np.pi * u.sr) #euclidean volume

    # convert euclidean to comving volume
    if cosmology:
        dV *= dVC_dVL_for_DL(r)
    
    V = np.sum(dV)

    print(f'{name} has total volume: {V} Mpc3') 

    return V, dV

In [24]:
voxels_gracedb = [get_skymap_voxels(data, name) for data,name in zip(skymap_gracedb, names_gracedb)]

S230518h has total volume: 23146839.76344147 Mpc3
S230731an has total volume: 30102502.05441636 Mpc3
S231113bw has total volume: 86159440.48744766 Mpc3
S230529ay has total volume: 1234160364.733098 Mpc3
S230627c has total volume: 4097930.0219590287 Mpc3


In [25]:
# check euclidean volumes to validate assumption comoving volume is necessary at these redshifts

voxels__gracedb_euc = [get_skymap_voxels(data, name, cosmology=False) for data,name in zip(skymap_gracedb, names_gracedb)]

S230518h has total volume: 46722498.16435014 Mpc3
S230731an has total volume: 60762683.431247875 Mpc3
S231113bw has total volume: 173915071.8265345 Mpc3
S230529ay has total volume: 2491184799.5262804 Mpc3
S230627c has total volume: 8271778.34578611 Mpc3


In [26]:
voxels_sim = [get_skymap_voxels(data, name) for data,name in zip(skymap_sim, names_sim)]

14 has total volume: 353968271.0017204 Mpc3
60 has total volume: 10364886.752256963 Mpc3


In [44]:
# combine event info to more easily access

events_gracedb = [[name, skymap, voxel] for name, skymap, voxel in zip(names_gracedb, skymap_gracedb, voxels_gracedb)]
events_sim = [[name, skymap, voxel] for name, skymap, voxel in zip(names_sim, skymap_sim, voxels_sim)]

# get rates from volumes

In [36]:
transient_rates = {
    'SNIa': 2.35e4,  # per Gpc^3 per year
    'CCSN': 1.01e5,
    'SLSN': 5.6,
    'KN': 5e3,
    'GRB_on_axis': 1,
    'GRB_off_axis': 7,
    'CV': 1e6,  # Use a refined rate near the galactic plane
}

In [65]:
# probability density function for radii of events evenly distributed in sphere
def pdf_radii(num_events, sphere_radius = 1000):
    """
    Sample radii for events evenly distributed in a sphere.
    """
    u = np.random.uniform(0, 1, num_events)
    radii = sphere_radius * np.cbrt(u)  # Inverse transform sampling for r^2 PDF
    return radii

# get ras and decs
def random_ra_dec_skymap(num_events, skymap, contour=0.9):
    """
    Given skymap and number of events, return random ra, dec positions in 90% region
    """
    skymap.sort('PROBDENSITY', reverse = True)
    level, ipix = ah.uniq_to_level_ipix(skymap['UNIQ']) # ipix = sky location
    nside = ah.level_to_nside(level) # nside = multi-order pixel resolution
    pixel_area = ah.nside_to_pixel_area(ah.level_to_nside(level)) # pixel area in steradians
    prob = pixel_area * skymap['PROBDENSITY']
    cumprob = np.cumsum(prob)
    i = cumprob.searchsorted(contour)

    #get ra and dec in degrees associated with each pixel
    # fix this, temporarily only taking discrete ra and dec associated with each pixel
    ra, dec = ah.healpix_to_lonlat(ipix[:i], nside[:i])
    skymap_ra_deg = [r.deg for r in ra]
    skymap_dec_deg = [d.deg for d in dec]
    ras = np.random.choice(skymap_ra_deg, num_events)
    decs = np.random.choice(skymap_dec_deg, num_events)
    return ras, decs

# get rates for each transient given a skymap
def get_rates(literature_rates, voxel, name, time_window=7, verbose=True):
    num_events = []
    volume = voxel[0] * 10**-9 # convert to Gpc^3
    for transient, rate in literature_rates.items():
        events = rate * volume * time_window / 365.25
        num_events.append(float(events))
    if verbose:
        print(name)
        for item1, item2 in zip(list(literature_rates.keys()), num_events):
            print(f"{item1} : {item2}")
    return num_events

In [66]:
rates_gracedb = [get_rates(transient_rates, voxel=event[2], name=event[0], time_window=7) for event in events_gracedb]

S230518h
SNIa : 10.424791625150231
CCSN : 44.80442358043291
SLSN : 0.0024842056638655872
KN : 2.21804077130856
GRB_on_axis : 0.000443608154261712
GRB_off_axis : 0.003105257079831984
CV : 443.608154261712
S230731an
SNIa : 13.55745814634221
CCSN : 58.26822437364098
SLSN : 0.0032307134306177173
KN : 2.884565563051534
GRB_on_axis : 0.0005769131126103067
GRB_off_axis : 0.004038391788272147
CV : 576.9131126103067
S231113bw
SNIa : 38.8041833269956
CCSN : 166.77542621389597
SLSN : 0.009246954324730865
KN : 8.256209218509701
GRB_on_axis : 0.00165124184370194
GRB_off_axis : 0.01155869290591358
CV : 1651.2418437019405
S230529ay
SNIa : 555.8367693322235
CCSN : 2388.9154767044497
SLSN : 0.13245471950044474
KN : 118.26314241111139
GRB_on_axis : 0.023652628482222278
GRB_off_axis : 0.16556839937555595
CV : 23652.628482222277
S230627c
SNIa : 1.845611194010295
CCSN : 7.932201301916587
SLSN : 0.0004398052207003256
KN : 0.3926832327681478
GRB_on_axis : 7.853664655362957e-05
GRB_off_axis : 0.00054975652587

In [67]:
rates_sim = [get_rates(transient_rates, voxel=event[2], name=event[0], time_window=7) for event in events_sim]

14
SNIa : 159.4189748933142
CCSN : 685.1624027329674
SLSN : 0.03798920252776849
KN : 33.91893082836472
GRB_on_axis : 0.006783786165672944
GRB_off_axis : 0.04748650315971061
CV : 6783.786165672946
60
SNIa : 4.668100946601698
CCSN : 20.062901940713687
SLSN : 0.0011123985234455112
KN : 0.9932129673620635
GRB_on_axis : 0.0001986425934724127
GRB_off_axis : 0.0013904981543068889
CV : 198.64259347241273


In [69]:
def get_transients(literature_rates,event,time_window=7):
    """
    get sample of transients 
    
    Parameters:
    transient_rates: dictionary of transient types and rates in Gpc^3 / year
    voxels: list of tuples containing volume, volume per pixel, ra, dec for a given skymap
    skymap: table containing skymap data
    time_window: time window in days to calculate expected transients

    returns: 
    number of expected transients for each transient type, with sampled ra, dec, and distance (change this to redshift)
    """

    # get number of events for each transient type  
    voxel=event[2]
    name=event[0]
    num_events = get_rates(literature_rates, voxel=voxel, name=name, time_window=time_window, verbose=False)
    
    #replace this with poisson sampling
    int_events = [round(num) for num in num_events]

    # sample ra and dec for each transients types
    skymap=event[1]
    radec = [random_ra_dec_skymap(num, skymap) for num in int_events]

    # get distances in Mpc from sphere with radius 1000Mpc
    dists = [pdf_radii(num) for num in int_events]
    
    return(num_events, radec, dists)

In [70]:
# test with one gracedb event

gracedb_test = get_transients(transient_rates, events_gracedb[0])

In [73]:
# test with one simulated event

gracedb_test = get_transients(transient_rates, events_sim[0])